I have editted the week 2 document that you had started and added in some things.

First things are needs for all weeks exercises.

In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Preamble 
import sys 
sys.path.append('..')

In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

df_train = train_set.to_pandas()
df_val = validation_set.to_pandas()

df_train.head()


,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
2,వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,వేప,telugu,"{'answer_start': [12], 'answer_text': ['Azadir...","వేప (లాటిన్ Azadirachta indica, syn. Melia aza...",https://te.wikipedia.org/wiki/%E0%B0%B5%E0%B1%...
3,চেঙ্গিস খান কোন বংশের রাজা ছিলেন ?,চেঙ্গিজ খান,bengali,"{'answer_start': [414], 'answer_text': ['বোরজি...",চেঙ্গিজ খান (মঙ্গোলীয়: Чингис Хаан আ-ধ্ব-ব: ...,https://bn.wikipedia.org/wiki/%E0%A6%9A%E0%A7%...
4,రెయ్యలగడ్ద గ్రామ విస్తీర్ణత ఎంత?,రెయ్యలగడ్ద,telugu,"{'answer_start': [259], 'answer_text': ['27 హె...","రెయ్యలగడ్ద, విశాఖపట్నం జిల్లా, గంగరాజు మాడుగుల...",https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B1%...


In [4]:
# Get train and validation data for each language
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

df_val_bengali = df_val[df_val['language'] == 'bengali']
df_val_arabic = df_val[df_val['language'] == 'arabic']
df_val_indonesian = df_val[df_val['language'] == 'indonesian']

print(len(df_train_bengali))
print(len(df_train_arabic))
print(len(df_train_indonesian))

# For testing
df_val_english = df_val[df_val['language'] == 'english']
df_train_english = df_train[df_train['language'] == 'english']


4779
29598
11394


In [5]:
df_train_bengali_document = df_train[df_train['language'] == 'bengali']["document_plaintext"]
df_train_arab_document = df_train[df_train['language'] == 'arabic']["document_plaintext"]
df_train_indonesian_document = df_train[df_train['language'] == 'indonesian']["document_plaintext"]
df_train_indonesian_document.head()

df_train_english_document = df_train[df_train['language'] == 'english']["document_plaintext"]


In [6]:
# Tokenize the documents
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def tokenize(df, key, transformer_model):
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]

# Tokinize train document_plaintext
tokenize(df_train_bengali, "document_plaintext", mbert_tokeniser)
tokenize(df_train_arabic, "document_plaintext", mbert_tokeniser)
tokenize(df_train_indonesian, "document_plaintext", mbert_tokeniser)

# Tokinize validation document_plaintext
tokenize(df_val_bengali, "document_plaintext", mbert_tokeniser)
tokenize(df_val_arabic, "document_plaintext", mbert_tokeniser)
tokenize(df_val_indonesian, "document_plaintext", mbert_tokeniser)


# For testing
tokenize(df_train_english, "document_plaintext", mbert_tokeniser)
tokenize(df_val_english, "document_plaintext", mbert_tokeniser)



Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors
/var/folders/96/51nz4vj53xj0q5n07407wbrr0000gq/T/com.apple.shortcuts.mac-helper/ipykernel_27449/3821323326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
/var/folders/96/51nz4vj53xj0q5n07407wbrr0000gq/T/com.apple.shortcuts.mac-helper/ipykernel_27449/3821323326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

I have moved tokenization to here, and # it out where it was before.

In [7]:
# added in tokenization of the questions
# Tokinize train question_text
tokenize(df_train_bengali, "question_text", mbert_tokeniser)
tokenize(df_train_arabic, "question_text", mbert_tokeniser)
tokenize(df_train_indonesian, "question_text", mbert_tokeniser)

# Tokinize validation question_text
tokenize(df_val_bengali, "question_text", mbert_tokeniser)
tokenize(df_val_arabic, "question_text", mbert_tokeniser)
tokenize(df_val_indonesian, "question_text", mbert_tokeniser)

/var/folders/96/51nz4vj53xj0q5n07407wbrr0000gq/T/com.apple.shortcuts.mac-helper/ipykernel_27449/3821323326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
/var/folders/96/51nz4vj53xj0q5n07407wbrr0000gq/T/com.apple.shortcuts.mac-helper/ipykernel_27449/3821323326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'{key}_tokenized'] = [transformer_model.tokenize(row) for row in df[key]]
/var/folders/96/51nz4vj5

In [8]:
# Training data document_plaintext tokenized
document_plaintext_tokenized_bengali = list(df_train_bengali["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_arabic = list(df_train_arabic["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_indonesian = list(df_train_indonesian["document_plaintext_tokenized"].explode())

# Validation data document_plaintext tokenized
document_plaintext_tokenized_val_bengali = list(df_val_bengali["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_val_arabic = list(df_val_arabic["document_plaintext_tokenized"].explode())
document_plaintext_tokenized_val_indonesian = list(df_val_indonesian["document_plaintext_tokenized"].explode())

In [9]:
# Training data question_text tokenized
question_text_tokenized_bengali = list(df_train_bengali["question_text_tokenized"].explode())
question_text_tokenized_arabic = list(df_train_arabic["question_text_tokenized"].explode())
question_text_tokenized_indonesian = list(df_train_indonesian["question_text_tokenized"].explode())

# Validation data question_text tokenized
question_text_tokenized_val_bengali = list(df_val_bengali["question_text_tokenized"].explode())
question_text_tokenized_val_arabic = list(df_val_arabic["question_text_tokenized"].explode())
question_text_tokenized_val_indonesian = list(df_val_indonesian["question_text_tokenized"].explode())

Week 2

Let k be the number of members in your group (k ∈ {1,2,3}). Implement k different7 language models for each of the three languages, separately for the questions and the documents (total k×3×2 language models), using the training data. Evaluate each of them on the validation data, report their performance and discuss the results.

First, models created for the document texts. This includes: UniformLM, Unigram, Unigram with Laplace smoothing, Bigram with Laplace smoothing and Trigram with Laplace smoothing.

Laplace smoothing was added to N-gram models where N>1 as an error occurred meaning that perplexity was infinite. **add explanation of why this was**

**We only need to keep 3 of the following 5 models.**

In [21]:
from statnlpbook.lm import NGramLM, UniformLM, inject_OOVs, perplexity, replace_OOVs, LaplaceLM

# DELETE - moved to above
# # Training data document_plaintext tokenized
# document_plaintext_tokenized_bengali = list(df_train_bengali["document_plaintext_tokenized"].explode())
# document_plaintext_tokenized_arabic = list(df_train_arabic["document_plaintext_tokenized"].explode())
# document_plaintext_tokenized_indonesian = list(df_train_indonesian["document_plaintext_tokenized"].explode())

# # Validation data document_plaintext tokenized
# document_plaintext_tokenized_val_bengali = list(df_val_bengali["document_plaintext_tokenized"].explode())
# document_plaintext_tokenized_val_arabic = list(df_val_arabic["document_plaintext_tokenized"].explode())
# document_plaintext_tokenized_val_indonesian = list(df_val_indonesian["document_plaintext_tokenized"].explode())

def perplexity_uniform(train, test):
    oov_train = inject_OOVs(train)
    oov_vocab = set(oov_train)
    oov_test = replace_OOVs(oov_vocab, test)
    oov_baseline = UniformLM(oov_vocab)
    return perplexity(oov_baseline,oov_test)

print("Bengali perplexity UniformLM:",perplexity_uniform(document_plaintext_tokenized_bengali, document_plaintext_tokenized_val_bengali))
print("Arabic perplexity UniformLM:",perplexity_uniform(document_plaintext_tokenized_arabic, document_plaintext_tokenized_val_arabic))
print("Indonesian perplexity UniformLM:",perplexity_uniform(document_plaintext_tokenized_indonesian, document_plaintext_tokenized_val_indonesian))


Bengali perplexity UniformLM: 3623.9999999845454
Arabic perplexity UniformLM: 11174.000000245973
Indonesian perplexity UniformLM: 18974.000000269545


In [22]:
def perplexity_ngram(train, test, n):
    oov_train = inject_OOVs(train)
    oov_vocab = set(oov_train)
    oov_test = replace_OOVs(oov_vocab, test)
    oov_baseline = NGramLM(oov_train, n)
    return perplexity(oov_baseline,oov_test)

print("Bengali perplexity Unigram:",perplexity_ngram(document_plaintext_tokenized_bengali, document_plaintext_tokenized_val_bengali, 1))
print("Arabic perplexity Unigram:",perplexity_ngram(document_plaintext_tokenized_arabic, document_plaintext_tokenized_val_arabic, 1))
print("Indonesian perplexity Unigram:",perplexity_ngram(document_plaintext_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 1))



Bengali perplexity Unigram: 371.19288552880323
Arabic perplexity Unigram: 751.8094261847106
Indonesian perplexity Unigram: 1735.9898871981518


In [23]:
# creating ngram model with laplace smoothing
def perplexity_ngram_laplace(train, test, n):
    oov_train = inject_OOVs(train)
    oov_vocab = set(oov_train)
    oov_test = replace_OOVs(oov_vocab, test)
    oov_baseline = LaplaceLM(NGramLM(oov_train, n), alpha=1) # alpha should be set to 1 as we are adding 1 word? 
    return perplexity(oov_baseline,oov_test)

# first comparing difference with adding laplace smoothing to unigram model
print("Bengali perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_bengali, document_plaintext_tokenized_val_bengali, 1))
print("Arabic perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_arabic, document_plaintext_tokenized_val_arabic, 1))
print("Indonesian perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 1))



Bengali perplexity Unigram with Laplace smoothing: 371.6411272453043
Arabic perplexity Unigram with Laplace smoothing: 751.5783613512319
Indonesian perplexity Unigram with Laplace smoothing: 1729.8410811023289


In [24]:
# BiGram with laplace smoothing, no longer has 'inf' bug/error 
print("Bengali perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_bengali, document_plaintext_tokenized_val_bengali, 2))
print("Arabic perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_arabic, document_plaintext_tokenized_val_arabic, 2))
print("Indonesian perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 2))

Bengali perplexity Bigram with Laplace smoothing: 161.7756039883504
Arabic perplexity Bigram with Laplace smoothing: 366.7034046492581
Indonesian perplexity Bigram with Laplace smoothing: 2021.0868388849685


In [25]:
print("Bengali perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_bengali, document_plaintext_tokenized_val_bengali, 3))
print("Arabic perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_arabic, document_plaintext_tokenized_val_arabic, 3))
print("Indonesian perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(document_plaintext_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 3))

Bengali perplexity Trigram with Laplace smoothing: 683.8866567672605
Arabic perplexity Trigram with Laplace smoothing: 2098.1074792562567
Indonesian perplexity Trigram with Laplace smoothing: 8404.6716095129


Next, models are created for the question texts for all 5 models as outlined above.

In [26]:
print("Bengali question perplexity UniformLM:",perplexity_uniform(question_text_tokenized_bengali, document_plaintext_tokenized_val_bengali))
print("Arabic question perplexity UniformLM:",perplexity_uniform(question_text_tokenized_arabic, document_plaintext_tokenized_val_arabic))
print("Indonesian question perplexity UniformLM:",perplexity_uniform(question_text_tokenized_indonesian, document_plaintext_tokenized_val_indonesian))

Bengali question perplexity UniformLM: 694.9999999976678
Arabic question perplexity UniformLM: 2578.0000000642076
Indonesian question perplexity UniformLM: 5437.999999995055


In [27]:
print("Bengali questions perplexity Unigram:",perplexity_ngram(question_text_tokenized_bengali, document_plaintext_tokenized_val_bengali, 1))
print("Arabic questions perplexity Unigram:",perplexity_ngram(question_text_tokenized_arabic, document_plaintext_tokenized_val_arabic, 1))
print("Indonesian questions perplexity Unigram:",perplexity_ngram(question_text_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 1))

Bengali questions perplexity Unigram: 425.35365857669797
Arabic questions perplexity Unigram: 1133.505444711856
Indonesian questions perplexity Unigram: 990.4566121951657


In [28]:
print("Bengali questions perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_bengali, document_plaintext_tokenized_val_bengali, 1))
print("Arabic questions perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_arabic, document_plaintext_tokenized_val_arabic, 1))
print("Indonesian questions perplexity Unigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 1))

Bengali questions perplexity Unigram with Laplace smoothing: 419.05258362912895
Arabic questions perplexity Unigram with Laplace smoothing: 1103.461062556527
Indonesian questions perplexity Unigram with Laplace smoothing: 945.9552934166811


In [29]:
print("Bengali questions perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_bengali, document_plaintext_tokenized_val_bengali, 2))
print("Arabic questions perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_arabic, document_plaintext_tokenized_val_arabic, 2))
print("Indonesian questions perplexity Bigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 2))

Bengali questions perplexity Bigram with Laplace smoothing: 263.6057476078464
Arabic questions perplexity Bigram with Laplace smoothing: 963.6267293858549
Indonesian questions perplexity Bigram with Laplace smoothing: 2012.680873575219


In [30]:
print("Bengali questions perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_bengali, document_plaintext_tokenized_val_bengali, 3))
print("Arabic questions perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_arabic, document_plaintext_tokenized_val_arabic, 3))
print("Indonesian questions perplexity Trigram with Laplace smoothing:",perplexity_ngram_laplace(question_text_tokenized_indonesian, document_plaintext_tokenized_val_indonesian, 3))

Bengali questions perplexity Trigram with Laplace smoothing: 454.84625413299256
Arabic questions perplexity Trigram with Laplace smoothing: 1954.7181123047
Indonesian questions perplexity Trigram with Laplace smoothing: 4219.730994551042
